<a href="https://colab.research.google.com/github/khalilDimassi/Datasci_Resources/blob/master/project_v0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q networkx nltk gensim PyMuPDF transformers huggingface_hub spark-nlp pyspark sentence-transformers levenshtein rake_nltk keybert rebel ultralytics==8.0.20 PyMuPDF roboflow --quiet pdf2image opencv-python
!sudo apt-get install poppler-utils

In [2]:
import fitz
import re
import rebel

from google.colab.patches import cv2_imshow
from pdf2image import convert_from_path
from ultralytics import YOLO
from PIL import Image
import cv2

from tqdm import tqdm
import networkx as nx
import pandas as pd
import numpy as np

import nltk
import nltk.corpus
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
autocorr_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
autocorr_model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large')

from transformers import T5Tokenizer, T5ForConditionalGeneration
summ_tokenizer = T5Tokenizer.from_pretrained('t5-base')
summ_model = T5ForConditionalGeneration.from_pretrained('t5-base')

from keybert import KeyBERT
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

from transformers import pipeline
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
rebel_model = AutoModelForSeq2SeqLM.from_pretrained('Babelscape/rebel-large')
rebel_tokenizer = AutoTokenizer.from_pretrained('Babelscape/rebel-large')

from pyspark.sql import SparkSession
from sparknlp.base import DocumentAssembler
from sparknlp.base import *
from sparknlp.annotator import SentenceDetectorDLModel
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.ml import Pipeline

spark = sparknlp.start()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please 

In [3]:
def extract_text_from_pdf(file_path, start=1, finish=None):
    """
    Extract text from a PDF document.

    Args:
        file_path (str): Path to the PDF file.
        start (int): The page number to start extraction (default is 1).
        finish (int, optional): The page number to finish extraction (default is None, which extracts until the last page).

    Returns:
        str: Extracted text from the specified pages.
    """
    with fitz.open(file_path) as doc:
        text = ''
        for i in range(start-1, finish or len(doc)):
            page = doc.load_page(i)
            text += page.get_text()
        return text

In [4]:
def extract_content(text, pattern):
  """
  Extract content based on a specified pattern from a text.

  Args:
      text (str): The text to extract content from.
      pattern (str): Regular expression pattern to match content.

  Returns:
      pd.DataFrame: A DataFrame containing extracted subjects and their associated content.
  """
  ListData = []
  matches = re.finditer(pattern, text)
  for match in matches:
    title = match.group(0).strip()
    start = match.start()
    end = match.end()
    content_start = end
    next_match = re.search(pattern, text[end:])
    if next_match:
      content_end = end + next_match.start()
    else:
      content_end = len(text)
    content = text[content_start:content_end].strip()
    couple = (title, content)
    ListData.append(couple)
  df = pd.DataFrame(ListData, columns=['subject', 'content'])
  return df


In [49]:
def clean_text(text):
    """
    Clean and preprocess text by removing non-ASCII characters, whitespace, punctuation, and digits.

    Args:
        text (str): Text to be cleaned.

    Returns:
        str: Cleaned text.
    """
    text = re.sub(r'\n397\n', '', text)
    text = re.sub(r'[^\x00-\x7F]+|\n|\t|[^\w\s]|\d', '', text)
    text = re.sub(r'uu', '', text)
    text = re.sub(r'Not For Distribution Sale or Reproduction', '', text)
    return text

In [6]:
def standardize_text(text):
    """
    Standardize text by converting it to lowercase.

    Args:
        text (str): Text to be standardized.

    Returns:
        str: Standardized text.
    """
    return text.lower()

In [7]:
def lemmatize_text(text):
    """
    Lemmatize text by reducing words to their base form.

    Args:
        text (str): Text to be lemmatized.

    Returns:
        str: Lemmatized text.
    """
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

In [8]:
def remove_stop_words(text, custom_stop_words=None):
    """
    Remove stop words from a list of words.

    Args:
        text (str): Text from which to remove stop words.
        custom_stop_words (list, optional): Custom stop words to include in addition to the default English stop words.

    Returns:
        str: Text with stop words removed.
    """
    words = text.split()
    stop_words = set(stopwords.words('english'))
    if custom_stop_words is not None:
        stop_words.update(custom_stop_words)
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = " ".join(filtered_words)
    return filtered_text



In [63]:
def clean(text):
    text = clean_text(text)
    text = standardize_text(text)
    text = standardize_text(text)
    text = lemmatize_text(text)
    text = remove_stop_words(text)
    return text

def clean_sentences_array(sentences_array):
    cleaned_sentences = []
    for sentence in sentences_array:
        cleaned_sentences.append(clean(sentence))
    return cleaned_sentences

def clean_dataframe(dataframe, columns):
    for column in columns:
        if column == 'sentences_array':
            dataframe[column] = dataframe[column].apply(clean_sentences_array)
        else:
            dataframe[column] = dataframe[column].apply(clean)

In [10]:
def calculate_token_length(subject, content, def_tokenizer):
    inputs = def_tokenizer.encode_plus(subject + ': ' + content, return_tensors='pt')
    return len(inputs['input_ids'][0])

In [11]:
def summarize_text(text, tokenizer, model):
    inputs = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0])

In [12]:
def extract_keywords(text, kw_model):
    keywords = kw_model.extract_keywords(text)
    return [keyword[0] for keyword in keywords]

In [24]:
def get_synonyms(word):
    synonyms = []
    for syn in nltk.corpus.wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return list(set(synonyms))  # Remove duplicates

In [34]:
def extract_triplets(sentence, tokenizer, model):
    inputs = tokenizer.encode(sentence, return_tensors='pt')
    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    triplets = tokenizer.decode(outputs[0])
    return triplets

In [83]:
def PDF_REMOVE_TXT(pdfTXT , pdfNoTXT):
    # Open the PDF document
    doc = fitz.open(pdfTXT)
    for page in doc:
        # Create a redaction annotation that covers the entire page
        rect = fitz.Rect(0, 0, page.rect.width, page.rect.height)
        page.add_redact_annot(rect)

    # Apply redactions to remove all text
    for page in doc:
        page.apply_redactions()

    # Save the modified PDF without text
    doc.save(pdfNoTXT)

    # Close the document
    doc.close()

In [84]:
def PDF_TO_IMG(frompath , topath):
  # Convert the PDF to a list of images (one image per page)
  images = convert_from_path(frompath)
  image_paths=[]
  # Iterate through the images
  for i, image in enumerate(images):
      # Save each image as a file (optional)
      i = i+1
      ii = topath + "/page_"+str(i)+".png"
      image_paths.append(ii)
      image.save(f'{topath}/page_{i}.png', 'PNG')
  return image_paths

In [ ]:
!mkdir /content/outputcutimage
def cutimage(pathimage , x, y, width, height , i) :

  # Load the image
  image = cv2.imread(pathimage)

  # Crop the image using the bounding box coordinates
  cropped_image = image[y:height, x:width]
  img = "/content/outputcutimage/figure"+str(i)+".png"
  cv2.imwrite(img, cropped_image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [91]:
def modeldetect(imagepath , i):
  modelyolo = YOLO("/content/drive/MyDrive/Colab Notebooks/modelfigbest.pt")
  image = Image.open(imagepath)
  image = np.asarray(image)
  results = modelyolo.predict(image)
  labels = ["figure"]
  if(len(results[0].boxes.boxes)>0):
    #  print("************************" + str(len(results[0].boxes.boxes)) + str(i))
     if(imagepath == imgpath[54]):
        for box in results[0].boxes.boxes.numpy():
            # Extract the box coordinates
            x_min, y_min, x_max, y_max, confidence, class_id = box
            # print(type(x_min))
            # print(x_min, y_min, x_max, y_max, confidence, class_id)
            cutimage(imagepath , int(x_min) , int(y_min) , int(x_max) , int(y_max) , i)
            i = i + 1
     else :
      # Extract the box coordinates
      if(len(results[0].boxes.boxes) > 1):
            x_min, y_min, x_max, y_max, confidence, class_id = results[0].boxes.boxes.numpy()[1]
      else :
            x_min, y_min, x_max, y_max, confidence, class_id = results[0].boxes.boxes.numpy()[0]
      # print(type(x_min))
      # print(x_min, y_min, x_max, y_max, confidence, class_id)
      if(confidence > 0,9):
            cutimage(imagepath , int(x_min) , int(y_min) , int(x_max) , int(y_max) , i)
            i = i + 1
  return i

In [93]:
def pathimage(text,path):
    # Define the regular expression pattern
    pattern = r'11\.\d+ [A-Z][^\n]*'
    # Define a regular expression pattern to match a sentence.
    sentence_pattern = r"(Figure|Table) \d+-\d+\.\s[\w\s, &():]+\n"
    listPath=[]

    # Find all matches
    matches = re.finditer(pattern, text)

    for match in matches:
        Chapterpath = []
        # Get the matched title
        title = match.group(0).strip()

        # Find the start and end positions of the matched title
        start = match.start()
        end = match.end()

        # Find the content following the matched title
        content_start = end
        next_match = re.search(pattern, text[end:])

        if next_match:
            # If a next match is found, extract its start position
            content_end = end + next_match.start()
        else:
            # If no next match is found, use the end of the text
            content_end = len(text)

        # Extract the content (paragraph) for this title
        content = text[content_start:content_end].strip()
        # Use re.search() to find the first sentence in the paragraph.
        mtt = re.finditer(sentence_pattern, content)

        for m in mtt:
          extracted_sentence = m.group(0)
          numerofig = int(extracted_sentence.split("\n")[0].split(".")[0].split("-")[1])
          if(numerofig == 1) :
            numerofig = 5

          elif(numerofig > 4 ):
            numerofig = numerofig + 1
          pathimg = path+"/figure"+str(numerofig)+".png"
          Chapterpath.append(pathimg)
        listPath.append(Chapterpath)
    return listPath

In [76]:
# MPBOK ch11:
pdf_file_path = '/content/drive/MyDrive/Colab Notebooks/PMBOK ch11.pdf'
extraction_start_page = 3
extraction_finish_page = None  # None means extract until the last page
pattern_to_extract = r'11\.\d+\.\d+.\d+ [A-Z][^\n]*'

text = extract_text_from_pdf(pdf_file_path, extraction_start_page, extraction_finish_page)
df = extract_content(text, pattern_to_extract)

# load df into sparkdf
spark_df = spark.createDataFrame(df)

documentAssembler = DocumentAssembler().setInputCol("content").setOutputCol("document")
sentenceDetectorDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en").setInputCols(["document"]).setOutputCol("sentences")

pipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetectorDL
])

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [77]:
model = pipeline.fit(spark_df)
result = model.transform(spark_df)
result_df = result.toPandas()
result_df['sentences_array'] = result_df['sentences'].apply(lambda x: [i['result'] for i in x])

In [78]:
workDF = result_df.copy()

In [79]:
workDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   subject          82 non-null     object
 1   content          82 non-null     object
 2   document         82 non-null     object
 3   sentences        82 non-null     object
 4   sentences_array  82 non-null     object
dtypes: object(5)
memory usage: 3.3+ KB


In [80]:
clean_dataframe(workDF, ['subject', 'content', 'sentences_array'])

In [ ]:
# predicate & object extraction
workDF['subject_tr'], workDF['predicate'], workDF['object'] = zip(*workDF['sentences_array'].apply(lambda x: [extract_triplets(sentence, rebel_tokenizer, rebel_model) for sentence in x]))

In [96]:
# Keywords extraction
workDF['keywords'] = workDF['content'].apply(lambda x: extract_keywords(x, kw_model))

In [97]:
# Synonyms extraction
workDF['synonyms'] = workDF['keywords'].apply(lambda x: [get_synonyms(word) for word in x])

In [ ]:
# apply summary on content
workDF['definitions'] = workDF['content'].apply(lambda x: summarize_text(x, summ_tokenizer, summ_model))

In [ ]:
# Remove specific tags
workDF['definitions'] = workDF['definitions'].apply(lambda x: re.sub(r'<pad>|</s>', '', x))

In [ ]:
# PDF to images
!mkdir /content/imagesrc
frompath = "/content/drive/MyDrive/Colab Notebooks/PMBOK ch11.pdf"
topath = "/content/imagesrc"
imgpath = PDF_TO_IMG(frompath , topath)

In [92]:
i = 1
for path in imgpath :
   i = modeldetect(path , i)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4

In [94]:
pptext = extract_text_from_pdf(pdf_file_path, extraction_start_page, extraction_finish_page)
listepathimage = pathimage(pptext,"/content/outputcutimage")
print(listepathimage)

[['/content/outputcutimage/figure2.png', '/content/outputcutimage/figure3.png', '/content/outputcutimage/figure4.png', '/content/outputcutimage/figure5.png', '/content/outputcutimage/figure6.png'], ['/content/outputcutimage/figure7.png', '/content/outputcutimage/figure8.png'], ['/content/outputcutimage/figure9.png', '/content/outputcutimage/figure10.png', '/content/outputcutimage/figure11.png'], ['/content/outputcutimage/figure12.png', '/content/outputcutimage/figure13.png', '/content/outputcutimage/figure15.png', '/content/outputcutimage/figure16.png'], ['/content/outputcutimage/figure17.png', '/content/outputcutimage/figure17.png', '/content/outputcutimage/figure18.png'], ['/content/outputcutimage/figure19.png', '/content/outputcutimage/figure20.png'], ['/content/outputcutimage/figure21.png', '/content/outputcutimage/figure22.png']]


In [ ]:
workDF